In [ ]:
#import package
import bs4
import requests
import pandas as pd
import geopandas as gpd
import random
import math
from geopy.distance import geodesic
import sqlalchemy as db
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import matplotlib

## Part1 Data Processing